In [ ]:
import os
import numpy as np
import glob
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 
from tensorflow.keras import layers
from time import perf_counter 

  

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
%cd "/content/drive/MyDrive/acc"
!pip install unrar
!unrar x "data.rar" 


In [ ]:
train_dir="/content/drive/MyDrive/acc/data/train"
val_dir="/content/drive/MyDrive/acc/data/val"

In [ ]:
classes = ['accident','car']

In [ ]:
BATCH_SIZE =100
IMG_SHAPE = 150
image_gen_train = ImageDataGenerator(rescale =1./255,horizontal_flip=True,zoom_range=0.5,rotation_range=45)


train_data_gen = image_gen_train.flow_from_directory(batch_size=BATCH_SIZE,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_SHAPE,IMG_SHAPE),
                                                     class_mode='sparse')
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=val_dir,
                                                  target_size=(IMG_SHAPE,IMG_SHAPE),
                                                  class_mode='sparse')
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)


In [ ]:
model = tf.keras.models.Sequential([
       tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_SHAPE,IMG_SHAPE, 3)),
        tf.keras.layers.MaxPooling2D(2, 2), 
         tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2), 
         tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),  
        tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),
     tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation="softmax")                         
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])
model.summary()

In [ ]:
epochs = 50

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE)))
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("/content/drive/MyDrive/acc/final_model.h5")


In [ ]:
# Modify 'test1.jpg' and 'test2.jpg' to the images you want to predict on
%cd "/content/drive/MyDrive/acc"
import cv2

from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import imutils
from PIL import Image
import time

success=1

model = tf.keras.models.load_model("final_model.h5")
model.build()
model.summary()

vid=cv2.VideoCapture('1.mp4')
start=time.time()

while success:
  success, frame =vid.read()
  if(type(frame) !='None'):
   
# load an image from file
image = load_img('mug.jpg', target_size=(224, 224))
    frame=imutils.rotate_bound(frame,90)
    frame= Image.fromarray(frame)
    frame=frame.resize((150,150),Image.NEAREST)
    frame =np.array(frame)
    frame=frame.astype(np.int)
    frame/=255.0
    pred_frame=np.expand_dims(frame,axis=0)
    label=model.predict(pred_frame)
    print(label[0])
    if(label[0][0]>label[0][1]):
      print(label[0])
      action_frames.append(frame_counter)
end=time.time()
print("hi")
print(end-start)




In [ ]:
%cd "/content/drive/MyDrive/acc"
import cv2
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import imutils
from PIL import Image
import time
from google.colab.patches import cv2_imshow

COLORS = [(150, 150, 0), (0, 150, 150)]

model = tf.keras.models.load_model("final_model.h5")
model.build()
classe=["ACCIDENT !","Everything good"]

cap = cv2.VideoCapture('1.mp4')
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
dim = (150,150)
out = cv2.VideoWriter('OutputVideo.mp4', fourcc, 30.0, dim)
starting_time = time.time()
frame_counter = 0
org = (10, 50)
font = cv2.FONT_HERSHEY_PLAIN

while True:
    ret, frame = cap.read()

    frame_counter += 1
    if ret == False:
        break
 
    
    x=frame
    x = cv2.resize(x, dim, interpolation=cv2.INTER_AREA)
    

   
    
    
 
    x =np.array(x)
    x=x.astype(np.float)
    x/=255.0
    pred_frame=np.expand_dims(x,axis=0)
    label=model.predict(pred_frame)
    print(label[0])
    print(label[0])
    if(label[0][0]>0.3):
      x=0
    else:
      x=1
    cv2.putText(frame, classe[x], org, font, 
                   fontScale, COLORS[x], 1, cv2.LINE_AA)
 

   
    
    out.write(frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
out.release()

cap.release()
cv2.destroyAllWindows()
print('done')
    
    
  
